In [10]:
import requests
from bs4 import BeautifulSoup
import bs4
import traceback
import re
import urllib
import os
import pandas as pd
import numpy as np
os.chdir("C:/Users/BENLAI/Desktop/毕业论文/Data/行政区域面积")
path = os.getcwd()

In [11]:
#将中文转化为url编码
def transToUrl(city_lst, provi_lst):
    try:
        urllst = []
        for i in range(int(len(city_lst)/2)):
            city_dict={
                'shengji' : city_lst[2*i] ,
                'diji' : city_lst[2*i+1],
                'xianji' : -1
            }
            urlencode=urllib.parse.urlencode(city_dict, encoding='gb2312')
            urllst.append(urlencode)
        for i in range(len(provi_lst)):
            provience_dict={
                'shengji': provi_lst[i],
                'diji':-1,
                'xianji' : -1
            }
            urlencode=urllib.parse.urlencode(provience_dict, encoding='gb2312')
            urllst.append(urlencode)
        return urllst        
    except:
        return 'False'

In [12]:
def getHTMLText(url, code = 'utf-8'):
    try:
        r = requests.get(url)
        r.raise_for_status()
#        r.encoding = code
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "False"

In [19]:
def getAreaInfo(urllst, Area_info_url, fpath):
    count = 0
    for j in range(len(urllst)):
        url = Area_info_url + urllst[j]
        html = getHTMLText(url)
        try:
            if html == "":
                print('html error')
                continue

            soup = BeautifulSoup(html, 'html.parser')
            AreaInfo = soup.find('table', attrs = {'class':'info_table'})
            flag = 100
            while AreaInfo == None:#异常判断
                soup = BeautifulSoup(html, 'html.parser')
                AreaInfo = soup.find('table', attrs = {'class':'info_table'})
                flag = flag - 1
                if flag < 0:
                    print('AreaInfo = None error'+ url)
                    break
                continue

            valueList = AreaInfo.find_all('td')
            num_cities = int(len(valueList)/7)
            #columns = ['地名', '驻地', '人口（万人）', '面积（平方千米）', '行政区划代码', '区号', '邮编']
            info_matrix = np.zeros([num_cities, 7], str)
            info_data = pd.DataFrame(info_matrix)
            for i in range(num_cities):
                for j in range(7):
                    info_data.iloc[i][j] = valueList[ i * 7 + j ].text   
            info_data.to_csv( fpath, mode='a', header=False, encoding = 'utf-8')
            
        except:
            count = count + 1
            print('\r当前进度: {: 2f}%'.format(count*100/len(urllst)),end = '')
            traceback.print_exc()
            continue

In [20]:
def main():
    #读取省与地级市简称
    city_lst = []
    city_fd = open( "city_lst.txt", "r" ) 
    for line in city_fd.readlines():
        city_lst.append(list(map(str,line.split(','))))
    for item in city_lst:
        city_lst = item       
    provi_lst = []
    provi_fd = open( "province_lst.txt", "r" ) 
    for line in provi_fd.readlines():
        provi_lst.append(list(map(str,line.split(','))))
    for item in provi_lst:
        provi_lst = item

    #利用函数获得url编码
    urllst = transToUrl(city_lst, provi_lst)
    Area_info_url = 'http://xzqh.mca.gov.cn/defaultQuery?'
    output_file = 'ChinaAreaInfo.csv'
    getAreaInfo(urllst, Area_info_url, output_file)

In [21]:
main()

AreaInfo = None errorhttp://xzqh.mca.gov.cn/defaultQuery?shengji=%B0%B2%BB%D5%CA%A1%28%CD%EE%29&diji=-1&xianji=-1
当前进度:  3.225806%

Traceback (most recent call last):
  File "<ipython-input-19-eccd8ca253c6>", line 23, in getAreaInfo
    valueList = AreaInfo.find_all('td')
AttributeError: 'NoneType' object has no attribute 'find_all'


AreaInfo = None errorhttp://xzqh.mca.gov.cn/defaultQuery?shengji=%B9%E3%CE%F7%D7%B3%D7%E5%D7%D4%D6%CE%C7%F8%28%B9%F0%29&diji=-1&xianji=-1
当前进度:  6.451613%

Traceback (most recent call last):
  File "<ipython-input-19-eccd8ca253c6>", line 23, in getAreaInfo
    valueList = AreaInfo.find_all('td')
AttributeError: 'NoneType' object has no attribute 'find_all'
